# Основы  динамического программирования 

**Наиболее длинная возрастающая подпоследовательность**


Дана последовательность целых чисел. Необходимо найти ее самую длинную строго возрастающую подпоследовательность.

Начнем решать задачу с начала — будем искать ответ, начиная с первых членов данной последовательности. Для каждого номера i будем искать наибольшую возрастающую подпоследовательность, оканчивающуюся элементом в позиции i. 

Пусть исходная последовательность хранится в массиве A. В массиве L будем записывать длины максимальных подпоследовательностей, оканчивающихся текущим элементом. 
Пусть мы нашли все $L_i$ для $1 <= i <= k – 1$. Теперь можно найти $L_k$ следующим образом. Просматриваем все элементы $A_i$ для $1 <= i < k – 1$ Если
$A_i < A_k$, то k-ый элемент может стать продолжением подпоследовательности, окончившейся элементом $A_i$. Длина полученной подпоследовательности будет на 1 больше $L_i$. Чтобы найти $L_k$, необходимо перебрать все i от 1 до k – 1:
$L_k = max(L_i) + 1$, где максимум берется по всем i таким, что $A_i < A_k$ и
$1 <= i < k$

In [1]:
import numpy as np

In [151]:
def get_the_longest_subsequence(A):
    
    """
    Вычисляем параметры последовательности для входного вектора A
    
    L: список максимальных подпоследовательностей A[i] элемента
    N: указатель на предыдущий элемент подпоследовательности
    """
    
    L = np.zeros(len(A))  # max len list
    N = np.zeros(len(A))  # last max elemnt pointer

    for i in range(len(A)):
        if i == 0:
            L[i] = 1
            N[i] = 0
            continue
        l_min = 0
        for j in range(len(A[:i])):
            if A[j] > l_min and A[j] < A[i]:
                L[i] += 1
                N[i] = j+1
                l_min = A[j]
        L[i] += 1        
    return L, N


def restore_the_sequence(A, N, L):
    
    """
    Восстановление подпоследовательности по указателю N
    """
    
    A_restored = []
    n = int(L.max())
    n_last = np.inf
    for i in range(n+1):
        if N[n-i] < n_last:
            A_restored.append(A[n-i])
            n_last = N[n-i]
    A_restored = A_restored[::-1]   
    return A_restored


def solve_sequrence(A):
    
    """
    Выводит результаты расчетов
    """
    
    L,N = get_the_longest_subsequence(A)
    AA = restore_the_sequence(A, N, L)
    print(f"A: {A}\n")
    print("Вычисляемые параметры:")
    print(f"N: {N}")
    print(f"L: {L}\n")
    print(f"Самая длинная возрастающая подпоследовательность: \n{AA}")

In [152]:
A = np.array([2,8,5,9,12, 6])  # data
solve_sequrence(A)

A: [ 2  8  5  9 12  6]

Вычисляемые параметры:
N: [0. 1. 1. 2. 4. 3.]
L: [1. 2. 2. 3. 4. 3.]

Самая длинная возрастающая подпоследовательность: 
[2, 5, 9, 12]


**Наибольшая общая подпоследовательность**

Рассмотрим последовательность чисел $a_1, a_2, …, a_n$. Если вычеркнуть из этой последовательности часть чисел, мы получим другую последовательность, которую называют подпоследовательностью данной последовательности.

Рассмотрим теперь еще одну последовательность $b_1, b_2, …, b_m$. Требуется найти длину самой длинной подпоследовательности последовательности a, которая одновременно является и подпоследовательностью последовательности b. Такую последовательность называют наибольшей общей подследовательностью (НОП). 

Например, для последовательностей (1, 2, 3, 4, 5) и (2, 7, 3, 2, 5) НОП является подпоследовательность 2, 3, 5, состоящая из трёх членов.



Опишем подзадачи, на которые мы будем разбивать нашу задачу. 
Мы напишем функцию $LCS(p, q)$, которая находит длину НОП для двух начальных участков $a_1, …, a_p$ и $b_1, …, b_q$ наших последовательностей. Пусть для всех пар q и p $(p < n, q < m)$, мы задачу решать уже научились.  Попробуем вычислить $LCS(n, m)$. Рассмотрим два случая:


1) $$           a_n= b_m.\space then \space LCS(n, m)=LCS(n-1, m-1)+1.$$

2) $$           a_n \not= b_m.\space then \space LCS(n, m)=max(LCS(n, m-1), LCS(n-1, m))$$.

Пользуясь этими формулами, мы можем заполнить таблицу значений LCS(p, q) для всех p и q последовательно: сначала заполняем первую строчку слева направо, затем вторую и т.д. Последнее число в последней строке и будет ответом на поставленную задачу.


Теперь заведем массив $F[n+1]$, который будем последовательно заполнять значениями выписанного рекуррентного соотношения. Будем предполагать, что количество номиналов банкнот хранится в переменной k, а сами номиналы хранятся в массиве $a[k]$.

In [162]:
def get_LCS_matrix(x, y):
    
    """
    Строит LCS матрицы ДП
    Использует алгоритм Нидлмана—Вунша
    """
    
    L = [[0]*(len(y)+1) for _ in range(len(x)+1)]
    for x_i,x_elem in enumerate(x):
        for y_i,y_elem in enumerate(y):
            if x_elem == y_elem:
                L[x_i][y_i] = L[x_i-1][y_i-1] + 1
            else:
                L[x_i][y_i] = max((L[x_i][y_i-1],L[x_i-1][y_i]))
    return L

def get_LCS_res(x, y):
    
    """
    Выводит LCS используя поиск по матрице
    """
    
    L = get_LCS_matrix(x, y)
    LCS = []
    x_i,y_i = len(x)-1,len(y)-1
    while x_i >= 0 and y_i >= 0:
        if x[x_i] == y[y_i]:
            LCS.append(x[x_i])
            x_i, y_i = x_i-1, y_i-1
        elif L[x_i-1][y_i] > L[x_i][y_i-1]:
            x_i -= 1
        else:
            y_i -= 1
    LCS.reverse()
    return LCS

In [ ]:
A = np.array([1,2,3,4,5])
B = np.array([2,7,3,2,5])

In [159]:
get_LCS_matrix(A, B)

[[0, 0, 0, 0, 0, 0],
 [1, 1, 1, 1, 1, 0],
 [1, 1, 2, 2, 2, 0],
 [1, 1, 2, 2, 2, 0],
 [1, 1, 2, 2, 3, 0],
 [0, 0, 0, 0, 0, 0]]

In [164]:
get_LCS_res(A,B)

[2, 3, 5]